Inspired by: https://huggingface.co/blog/train-sentence-transformers

In [ ]:
import os
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, SentenceTransformerTrainingArguments
from sentence_transformers.losses import CoSENTLoss, MultipleNegativesRankingLoss, SoftmaxLoss
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, TripletEvaluator, LabelAccuracyEvaluator, SequentialEvaluator
from transformers import EarlyStoppingCallback
import wandb

# Set environment variables
os.environ["NCCL_P2P_DISABLE"] = "1"
os.environ["NCCL_IB_DISABLE"] = "1"

# Initialize W&B project
wandb.login(key=os.getenv('WANDB_API_TOKEN'))

base_model_username = 'allenai'
base_model_name = 'longformer-base-4096'

# 1. Load a model to finetune
model = SentenceTransformer(f"{base_model_username}/{base_model_name}")

# 2. Load datasets for training and evaluation
train_dataset = {
    "all-nli-pair": load_dataset("sentence-transformers/all-nli", "pair", split="train[:10000]"),
    "all-nli-pair-class": load_dataset("sentence-transformers/all-nli", "pair-class", split="train[:10000]"),
    "all-nli-pair-score": load_dataset("sentence-transformers/all-nli", "pair-score", split="train[:10000]"),
    "all-nli-triplet": load_dataset("sentence-transformers/all-nli", "triplet", split="train[:10000]"),
    "stsb": load_dataset("sentence-transformers/stsb", split="train[:10000]"),
    "quora": load_dataset("sentence-transformers/quora-duplicates", "pair", split="train[:10000]"),
    "natural-questions": load_dataset("sentence-transformers/natural-questions", split="train[:10000]"),
}

eval_dataset = {
    "all-nli-triplet": load_dataset("sentence-transformers/all-nli", "triplet", split="dev"),
    "stsb": load_dataset("sentence-transformers/stsb", split="validation"),
    "quora": load_dataset("sentence-transformers/quora-duplicates", "pair", split="train[10000:11000]"),
    "natural-questions": load_dataset("sentence-transformers/natural-questions", split="train[10000:11000]"),
}

# Define loss functions
embedding_dim = model.get_sentence_embedding_dimension()
num_labels = len(set(train_dataset["all-nli-pair-class"]["label"]))
mnrl_loss = MultipleNegativesRankingLoss(model)
softmax_loss = SoftmaxLoss(model, sentence_embedding_dimension=embedding_dim, num_labels=num_labels)
cosent_loss = CoSENTLoss(model)

losses = {
    "all-nli-pair": mnrl_loss,
    "all-nli-pair-class": softmax_loss,
    "all-nli-pair-score": cosent_loss,
    "all-nli-triplet": mnrl_loss,
    "stsb": cosent_loss,
    "quora": mnrl_loss,
    "natural-questions": mnrl_loss,
}

# Define evaluators for each dataset
stsb_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=eval_dataset["stsb"]["sentence1"],
    sentences2=eval_dataset["stsb"]["sentence2"],
    scores=eval_dataset["stsb"]["score"],
    name="sts-dev",
)

all_nli_triplet_evaluator = TripletEvaluator(
    anchors=eval_dataset["all-nli-triplet"]["anchor"],
    positives=eval_dataset["all-nli-triplet"]["positive"],
    negatives=eval_dataset["all-nli-triplet"]["negative"],
    name="triplet-dev",
)

# Create EmbeddingSimilarityEvaluator for classification tasks
label_accuracy_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=train_dataset["all-nli-pair-class"]["premise"],  # Premises
    sentences2=train_dataset["all-nli-pair-class"]["hypothesis"],  # Hypotheses
    scores=train_dataset["all-nli-pair-class"]["label"],  # Ground-truth labels
    name="label-accuracy-dev",
)


# Combine evaluators into a SequentialEvaluator
evaluator = SequentialEvaluator(
    [stsb_evaluator, all_nli_triplet_evaluator, label_accuracy_evaluator],
    main_score_function=lambda scores: scores[-1] if scores else 0,
)

# Define custom aggregate metric
def compute_aggregate_metric(metrics):
    """
    Combine metrics from multiple evaluators into a single score for optimization.
    """
    stsb_score = metrics.get("eval_sts-dev_spearman_cosine", 0)
    triplet_score = metrics.get("eval_triplet-dev_mean_rank", 0)  # Replace with actual triplet metric key
    label_accuracy_score = metrics.get("eval_label-accuracy-dev_accuracy", 0)

    # Weighted average of metrics
    aggregate_score = (
        0.4 * stsb_score +
        0.4 * triplet_score +
        0.2 * label_accuracy_score
    )
    return aggregate_score

# Define W&B sweep configuration
sweep_config = {
    "method": "bayes",
    "metric": {"name": "aggregate_score", "goal": "maximize"},
    "parameters": {
        "learning_rate": {"min": 2e-5, "max": 5e-4},
        "warmup_steps": {"values": [0.1, 0.2, 0.3]},
    },
}

# Initialize W&B sweep
sweep_id = wandb.sweep(sweep_config, project="sentence-transformers-sweep")

# Define training function
def train_model(config=None):
    with wandb.init(config=config):
        config = wandb.config

        # Define training arguments
        total_steps = len(train_dataset)
        warmup_steps = int(config.warmup_steps * total_steps)

        training_args = SentenceTransformerTrainingArguments(
            output_dir=f"./{base_model_name}-fine-tuned",
            overwrite_output_dir=True,
            evaluation_strategy="steps",
            eval_steps=500,
            save_total_limit=3,
            load_best_model_at_end=True,
            metric_for_best_model="aggregate_score",  # Use custom metric here
            greater_is_better=True,
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            num_train_epochs=10,
            warmup_steps=warmup_steps,
            learning_rate=config.learning_rate,
            logging_dir="./logs",
            logging_steps=200,
            save_steps=500,
        )

        early_stopping = EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.01)

        # Define SequentialEvaluator
        evaluator = SequentialEvaluator(
            [stsb_evaluator, all_nli_triplet_evaluator, label_accuracy_evaluator],
            main_score_function=lambda scores: scores[-1] if scores else 0,
        )

        # Train model
        trainer = SentenceTransformerTrainer(
            model=model,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            loss=losses,
            args=training_args,
            evaluator=evaluator,
            callbacks=[early_stopping],
        )

        # Save a reference to the original evaluate method
        original_evaluate = trainer.evaluate

        # Wrap evaluation to include custom aggregate metric
        def custom_evaluate(*args, **kwargs):
            metrics = original_evaluate(*args, **kwargs)  # Call the original evaluate method
            # Calculate the aggregate score
            aggregate_score = compute_aggregate_metric(metrics)
            metrics["aggregate_score"] = aggregate_score
            wandb.log({"aggregate_score": aggregate_score})
            return metrics

        # Override the trainer's evaluate function
        trainer.evaluate = custom_evaluate

        trainer.train()

        # Save the model
        model.save_pretrained(f"{base_model_name}-best-model")
        wandb.save(f"{base_model_name}-best-model/*")



# Start W&B agent for the sweep
wandb.agent(sweep_id, function=train_model)

# Push the best model to the Hugging Face Hub with W&B config
model.push_to_hub(
    f"{base_model_name}-sentence-transformers-best",
    exist_ok=True,
)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: leo1212 (hslu_nlp). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/leonkrug/.netrc
No sentence-transformers model found with name allenai/longformer-base-4096. Creating a new one with mean pooling.


Create sweep with ID: mbg5ckdp
Sweep URL: https://wandb.ai/hslu_nlp/sentence-transformers-sweep/sweeps/mbg5ckdp


wandb: Agent Starting Run: 8yu15rc5 with config:
wandb: 	learning_rate: 0.00045788302667372136
wandb: 	warmup_steps: 0.3


/home/leonkrug/anaconda3/envs/aicomp/lib/python3.11/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Currently using DataParallel (DP) for multi-gpu training, while DistributedDataParallel (DDP) is recommended for faster training. See https://sbert.net/docs/sentence_transformer/training/distributed.html for more information.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'warmup_steps' was locked by 'sweep' (ignored update).


  0%|          | 0/41100 [00:00<?, ?it/s]

...

Traceback (most recent call last):
  File "/tmp/ipykernel_8990/1301178606.py", line 179, in train_model
    trainer.train()
  File "/home/leonkrug/anaconda3/envs/aicomp/lib/python3.11/site-packages/transformers/trainer.py", line 2052, in train
    return inner_training_loop(
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/leonkrug/anaconda3/envs/aicomp/lib/python3.11/site-packages/transformers/trainer.py", line 2388, in _inner_training_loop
    tr_loss_step = self.training_step(model, inputs)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/leonkrug/anaconda3/envs/aicomp/lib/python3.11/site-packages/transformers/trainer.py", line 3485, in training_step
    loss = self.compute_loss(model, inputs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/leonkrug/anaconda3/envs/aicomp/lib/python3.11/site-packages/sentence_transformers/trainer.py", line 344, in compute_loss
    loss = loss_fn(features, labels)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/leonkrug/ana

Run 0d6ir904 errored:
Traceback (most recent call last):
  File "/home/leonkrug/anaconda3/envs/aicomp/lib/python3.11/site-packages/wandb/agents/pyagent.py", line 306, in _run_job
    self._function()
  File "/tmp/ipykernel_8990/1301178606.py", line 179, in train_model
    trainer.train()
  File "/home/leonkrug/anaconda3/envs/aicomp/lib/python3.11/site-packages/transformers/trainer.py", line 2052, in train
    return inner_training_loop(
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/leonkrug/anaconda3/envs/aicomp/lib/python3.11/site-packages/transformers/trainer.py", line 2388, in _inner_training_loop
    tr_loss_step = self.training_step(model, inputs)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/leonkrug/anaconda3/envs/aicomp/lib/python3.11/site-packages/transformers/trainer.py", line 3485, in training_step
    loss = self.compute_loss(model, inputs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/leonkrug/anaconda3/envs/aicomp/lib/python3.11/site-packa

model.safetensors:   0%|          | 0.00/595M [00:00<?, ?B/s]

'https://huggingface.co/Leo1212/longformer-base-4096-sentence-transformers-best/commit/dce6d431c10e096140bece902bd3ee75d54e20a1'

: 